In [1]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
!pip uninstall transformers -y
!pip install transformers==4.41.0

In [ ]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
import transformers

In [4]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

In [ ]:
from huggingface_hub import login
login(token = 'token_here')

In [40]:
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ", use_fast=False)
config = PeftConfig.from_pretrained("DariaaaS/fanfiction-largedataset-test-ft")

base_model = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ", device_map = 'cuda')
model = PeftModel.from_pretrained(base_model, "DariaaaS/fanfiction-largedataset-test-ft")

In [30]:
def generate_text(prompt, model, tokenizer):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=1000)
    result = tokenizer.batch_decode(outputs)[0]
    result = result.replace(prompt, "")
    return result

def clean_text(generated_text: str):
    cleaned_text = re.sub(r'http\S+', '', generated_text)
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

def split_text_into_sections(text, delimiter='Chapter'):
    sections = [delimiter + x for x in text.split(delimiter) if x]
    return sections[1:]

def renumber_chapters_simple(list):
    renumbered_chapters = []
    for i, chapter in enumerate(list, 1):
        chapter_body = ' '.join(chapter.split(' ')[2:])
        renumbered_chapters.append(f"Chapter {i} {chapter_body}")
    return renumbered_chapters

def renumber_parts(list):
    renumbered_chapters = []
    for i, chapter in enumerate(list, 1):
        chapter_body = ' '.join(chapter.split(' ')[2:])
        renumbered_chapters.append(f"Part {i} {chapter_body}")
    return renumbered_chapters

def split_into_parts(text, delimiter='Part '):
    parts = [delimiter + x for x in text.split(delimiter) if x]
    return parts[1:]

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer_ = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ", device_map={"":0})
model_not_finetuned = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ", device_map={"":0})

In [8]:
zachini = open('text4.txt', 'r')
zachini = zachini.read()

# this prompt generates the first part of the outline for the future story
generate_outline = (f'Imagine You are a famous writer and you are planning to write a new book about Harry Potter.\
                Now you must use the "stuff" as a SOURCE of ideas to create a very detailed outline for a future story about Harry Potter. "Stuff": {zachini}.\
                Divide your outline into FIVE logical parts.\
                You must ALWAYS use this template: "Outline: Part №: *description of this part*"\
                Do not add comments.')

outline = generate_text(generate_outline, model_not_finetuned, tokenizer_)
delimiter = 'Outline'
outline_prepared  = [delimiter+x for x in outline.split(delimiter) if x]
outline_prepared = outline_prepared[1:]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [9]:
outline_prepared

["Outline: Part 1: Introduction to Theo and Pansy\n* Description of Theo's background and his excitement about attending Hogwarts\n* Description of Pansy's background and her desire for wealth and perfection\n* Their shared background as pure-blood wizards and their placement in Slytherin House\n* Their early interests and excelling in certain subjects\n\n",
 "Outline: Part 2: Growing Up in Slytherin\n* Description of the bullying they faced due to their pure-blood status and appearance\n* Theo's development of non-verbal spells and Pansy's struggles with Transfiguration\n* Their growing friendship and mutual respect\n\n",
 "Outline: Part 3: Preparing for O.W.L. Exams\n* Description of their dedication to their studies and their successes\n* Theo's disinterest in relationships and Pansy's longing for Draco Malfoy\n* Their occasional interactions and support for each other\n\n",
 'Outline: Part 4: The O.W.L. Exams and Success\n* Description of their preparation and taking the exams\n* T

In [ ]:
# this prompt generates the second part of the outline, so our story could be longer

generate_outline_continuation = (f'Imagine You are a famous writer and you are planning to write a continuation for the book about Harry Potter.\
                Look at the existing outline: {outline_prepared}. What will happen next? \
                Use the provided "stuff" as a SOURCE of ideas to create a continuation for the existing outline for a future story about Harry Potter. "Stuff": {zachini}.\
                Divide your outline into FIVE logical parts.\
                You must ALWAYS use this template: "Outline (continuation): Part №: *description of this part*".\
                Do not add comments.')
outline2 = generate_text(generate_outline_continuation, model_not_finetuned, tokenizer_)

In [17]:
generate_outline_continuation2 = (f'Imagine You are a famous writer and you are planning to write a continuation for the book about Harry Potter.\
                Look at the existing outline: {outline_2}. What will happen next? \
                Use the provided "stuff" as a SOURCE of ideas to create a continuation for the existing outline for a future story about Harry Potter. "Stuff": {zachini}.\
                Divide your outline into FIVE logical parts.\
                You must ALWAYS use this template: "Outline (continuation 2): Part №: *description of this part*".\
                Do not add comments.')
outline3 = generate_text(generate_outline_continuation2, model_not_finetuned, tokenizer_)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
delimiter = 'Outline (continuation)'
outline__2  = [delimiter+x for x in outline2.split(delimiter) if x]
outline__2 = outline__2[1:]


delimiter = 'Outline (continuation 2)'
outline__3  = [delimiter+x for x in outline3.split(delimiter) if x]
outline__3 = outline__3[1:]
outline__3

In [ ]:
generate_outline_continuation3 = (f'Imagine You are a famous writer and you are planning to write a continuation for the book about Harry Potter.\
                Look at the existing outline: {outline__3}. What will happen next? \
                Use the provided "stuff" as a SOURCE of ideas to create a continuation for the existing outline for a future story about Harry Potter. "Stuff": {zachini}.\
                Divide your outline into FIVE logical parts.\
                You must ALWAYS use this template: "Outline (continuation 3): Part №: *description of this part*".\
                Do not add comments.')
outline4 = generate_text(generate_outline_continuation3, model_not_finetuned, tokenizer_)
delimiter = 'Outline (continuation 3)'
outline_4  = [delimiter+x for x in outline4.split(delimiter) if x]
outline_4 = outline_4[1:]

generate_outline_continuation4 = (f'Imagine You are a famous writer and you are planning to write a continuation for the book about Harry Potter.\
                Look at the existing outline: {outline_4}. What will happen next? \
                Use the provided "stuff" as a SOURCE of ideas to create a continuation for the existing outline for a future story about Harry Potter. "Stuff": {zachini}.\
                Divide your outline into FIVE logical parts.\
                You must ALWAYS use this template: "Outline (continuation 4): Part №: *description of this part*".\
                Do not add comments.')
outline5 = generate_text(generate_outline_continuation4, model_not_finetuned, tokenizer_)
delimiter = 'Outline (continuation 4)'
outline_5  = [delimiter+x for x in outline5.split(delimiter) if x]
outline_5 = outline_5[1:]
outline_4
outline_5

In [ ]:
generate_outline_continuation5 = (f'Imagine You are a famous writer and you are planning to write a continuation for the book about Harry Potter.\
                Look at the existing outline: {outline_5}. What will happen next? \
                Use the provided "stuff" as a SOURCE of ideas to create a continuation for the existing outline for a future story about Harry Potter. "Stuff": {zachini}.\
                Divide your outline into FIVE logical parts.\
                You must ALWAYS use this template: "Outline (continuation 5): Part №: *description of this part*".\
                Do not add comments.')
outline6 = generate_text(generate_outline_continuation5, model_not_finetuned, tokenizer_)
delimiter = 'Outline (continuation 3)'
outline_6  = [delimiter+x for x in outline6.split(delimiter) if x]
outline_6 = outline_6[1:]

generate_outline_continuation6 = (f'Imagine You are a famous writer and you are planning to write a continuation for the book about Harry Potter.\
                Look at the existing outline: {outline_6}. What will happen next? \
                Use the provided "stuff" as a SOURCE of ideas to create a continuation for the existing outline for a future story about Harry Potter. "Stuff": {zachini}.\
                Divide your outline into FIVE logical parts.\
                You must ALWAYS use this template: "Outline (continuation 6): Part №: *description of this part*".\
                Do not add comments.')
outline7 = generate_text(generate_outline_continuation6, model_not_finetuned, tokenizer_)
delimiter = 'Outline (continuation 6)'
outline_7  = [delimiter+x for x in outline7.split(delimiter) if x]
outline_7 = outline_7[1:]
outline_6
outline_7

In [ ]:
final_outline = outline_prepared + outline__2 + outline__3 + outline_4 + outline_5 + outline_6 + outline_7
final_outline = ' '.join(final_outline)

In [32]:
delimiter = 'Part'
outline_parts = [delimiter+x for x in final_outline.split(delimiter) if x]
outline_parts = outline_parts[1:]
clean_outline = []
# for i in outline_parts:
#   outline_parts = outline_parts.replace('Outline', '').replace('Outline (continuation', '')
#   clean_outline.appendoutline_parts
outline_parts = renumber_parts(outline_parts)

In [33]:
create_chapters = []

for i in outline_parts:
    p1 = f'### Instruction:\nImagine You are a famous writer. You are writing a fantastic and magical fanfiction about Harry Potter FOR KIDS.  For your book you can only use the ideas from +\
          the original "Harry Potter" books and from the "stuff". ###stuff: {zachini}. \
          Now You are going to create a list of chapters for the first part of the book, according to the "outline". ### Outline: {i}.\
          You must use this template: Part №: "name of the part". Chapter №: *plot of this chapter*. \
          Create a list of five chapters!!!\n\n### List of chapters:'
    create_chapters.append(p1)
len(create_chapters)

34

In [34]:
import re

In [ ]:
chapters_list = ''
for i in create_chapters:
  res = generate_text(i, model_not_finetuned, tokenizer_)
  print(res)
  chapters_list += res
chapters_list = clean_text(chapters_list)
chapters = split_text_into_sections(chapters_list)
chapters = renumber_chapters_simple(chapters)

In [39]:
chapters

['Chapter 1 Theos Arrival at Hogwarts Theo eagerly awaits his Hogwarts letter finally receiving it and being sorted into Slytherin House He is thrilled with his new surroundings and begins to excel in Potions and Transfiguration ',
 'Chapter 2 Pansys Perfection Pansy grows up in a wealthy pureblood family determined to maintain her familys reputation and her own perfection She excels in Ancient Runes and Divination and sets her sights on Draco Malfoy ',
 'Chapter 3 Theos Magic Mastery Theo continues to hone his magical skills particularly in nonverbal Transfiguration He impresses Pansy with his ability to turn objects into various forms and she begins to take notice of him ',
 'Chapter 4 Pansys Desire for Wealth Pansys obsession with wealth and perfection leads her to seek out ways to increase her familys fortune She becomes involved in a dangerous scheme but Theos help saves her from disaster ',
 'Chapter 5 Theo and Pansys First Encounter Theo and Pansy meet for the first time on the 

In [ ]:
chapters_for_book = chapters
chapters_for_book

In [ ]:
### For generating the text of fanfiction itself we use our finetuned model, which
### was initialized at the beginning :)

model.eval()
text2 = ''
for i in chapters_for_book:
    prompt = f'### Instruction:\nImagine you are a teenage girl who adores books about Harry Potter. \
          You want to write a magical and fantastic fanfiction about Harry Potter. You want it to be detailed, interesting and long. !!!It must be a content for kids!!! \n\
          You must write only the CHAPTER provided in the INPUT. ### Input:\n{i}\n\n### Fanfiction:'
    final_chapter = generate_text(prompt, model, tokenizer)
    print(final_chapter)
    text2 += final_chapter

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
Chapter 1: Theos' Arrival at Hogwarts

Theos sat at the kitchen table, his eyes fixed on the fireplace. Every year since he could remember, he had watched as the other children in his village received their Hogwarts letters. Every year, he had felt a pang of longing, a deep desire to join them in their magical adventures.

But this year was different. This year, Theos had received a letter of his own. He had been up all night, his heart pounding in his chest as he read the words that would change his life forever.

"Welcome to Hogwarts, Theos. We are pleased to offer you a place in our esteemed institution of learning. We have placed you in Slytherin House."

Theos couldn't believe it. He had always known he was destined for something more than this quiet village life. And now, he was going to Hogwarts, the most magical place on earth.

The journey to Hogwarts was filled with wonder and excitement for Theos. He marveled at the floating candles that lit his way through the forest, 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
Title: The Enchanted Quill

Chapter 2: Pansy's Perfection

Pansy Parkinson was a name synonymous with perfection. Born into a wealthy and prestigious pureblood family, the Parkinsons took great pride in maintaining their impeccable reputation. Their ancestral home, located in the heart of the magical community, was a testament to their wealth and status. The grand mansion was adorned with enchanted flowers that bloomed all year round, and the gardens were home to rare and exotic creatures.

Pansy, the eldest of three children, was a prodigy. She excelled in every subject at Hogwarts, but her true passion lay in Ancient Runes and Divination. Her teachers were always impressed by her natural talent and dedication to her studies. Pansy was determined to uphold her family's legacy and become the perfect student.

As she grew older, Pansy's attention was drawn to one particular student at Hogwarts - Draco Malfoy. With his golden hair and piercing green eyes, Draco was the epitome of Ma

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

<s> 
---
Chapter 165: The Unexpected Revelation

As they worked together to decipher the ancient prophecy, Theo and Pansy found themselves in a rhythm. Their minds were in sync, their wands moving in perfect harmony as they traced the intricate lines of the parchment. The air around them crackled with magic, and the room seemed to hum with their combined efforts.

Theo, a tall and lanky boy with a mop of curly brown hair, looked over at Pansy, her long, dark hair cascading down her back like a waterfall. She was focused, her bright green eyes fixed on the parchment, her lips moving as she whispered the incantations. Theo felt a strange connection to her, a sense of understanding that went beyond their shared love for magic.

Pansy, who had always been the popular girl, the one with the perfect Slytherin uniform and the most elegant wand, looked up from the parchment and met Theo's gaze. She saw the determination in his eyes, the way his brow furrowed as he concentrated. And in that mom

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
Chapter 166: The Invitation to Hogwarts

Theo sat at the kitchen table, his eyes fixed on the owl perched on the windowsill. The sun was setting, casting a warm glow over the garden. He had been waiting for hours, his heart pounding with excitement. Finally, the owl spread its wings and took off into the twilight sky.

Theo watched as it flew away, then turned to his mother. "It's gone, Mum. Hogwarts has sent its invitation."

His mother smiled warmly. "I'm so proud of you, Theo. I know you'll do great things at Hogwarts."

Theo's father looked up from the newspaper. "You'll need to pack some things for school, son. And don't forget your wand."

Theo nodded, his mind racing with excitement. He had always known that he was destined for great things. He had grown up hearing stories of Harry Potter and his friends, and he longed to join them in their magical adventures.

The next day, Theo boarded the Hogwarts Express with a mixture of nervousness and excitement. He had heard stories

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
Title: The Magical Friendship of Theo and Pansy

Chapter 167: The Slytherin Bond

The sun was setting over Hogwarts Castle as Theo and Pansy made their way to the Slytherin common room. They had spent hours in the library, engrossed in their favorite books about Potions and Transfiguration. Theo, a tall, lanky boy with a mop of curly brown hair, and Pansy, a petite girl with long, dark hair and emerald green eyes, had become fast friends.

As they entered the common room, they were greeted by the familiar sight of green and silver banners hanging from the walls. Theo looked over at Pansy and smiled. "Another great day of learning, huh?" he asked, his voice filled with excitement.

Pansy nodded, her eyes shining with the same enthusiasm. "Absolutely! I can't get enough of Potions and Transfiguration. They're my favorite subjects," she replied, her voice barely above a whisper.

Theo chuckled. "I know the feeling. I've spent hours in the library, trying to learn every little detail 

In [39]:
import re

In [43]:
to_delete = '''The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.'''
text2 = fanfic.replace(to_delete, '')

In [53]:
text2 = text2.replace('\n', '').replace('<s>', '').replace('</s>', '')
pattern = re.search(r'(?<=\### Instruction).*(?=\### Fanfiction)', text2, re.DOTALL).group().strip() if re.search(r'(?<=\### Instruction).*(?=\### Fanfiction)', text2, re.DOTALL) else None
fanfic_formatted = text2.replace(pattern, '')
book = fanfic_formatted.replace('### Instruction', '').replace('### Fanfiction', '')
book

'Chapter 1: The Excited LetterTheo Nott, a thirteen-year-old boy with a mop of curly brown hair and bright green eyes, sat at the kitchen table, his heart pounding with excitement. The first of September had arrived, and with it, the long-awaited day when he would receive his Hogwarts letter.Theo\'s parents, both Muggles, had always been supportive of their son\'s love for books about magic and the fantastical world of Harry Potter. They had even bought him a beautiful, old oak desk for his birthday, which now stood proudly in his room, adorned with quills, ink, and an endless supply of parchment.As Theo sipped his morning tea, he couldn\'t help but daydream about the magical world that awaited him. He imagined himself as a Gryffindor student, with a scarf of that famous house\'s colors around his neck, flying on a broomstick over the castle towers, and making friends with magical creatures.Suddenly, there was a knock at the door. Theo\'s heart skipped a beat. Could it be? He jumped up

In [54]:
formatted_text = book.replace('\' ', '\'\n\n').replace('" ', '"\n\n').replace(' Chapter ', '\n\nChapter ').replace(' Mr.', '\n\nMr.').replace(' Mrs.', '\n\nMrs.').replace(' Prof.', '\n\nProf.')

formatted_file_path = 'baseline_formatted.txt'
with open(formatted_file_path, 'w') as file:
    file.write(formatted_text)

formatted_file_path

with open(formatted_file_path, 'r') as file:
    formatted_text = file.read()

lines = formatted_text.split('\n')

cleaned_lines = []
for line in lines:

    cleaned_line = ''.join([char for i, char in enumerate(line) if not (char == "'" and (i == 0 or line[i - 1] != '\\'))])


    if len(cleaned_line.strip()) >= 10:
        cleaned_lines.append(cleaned_line)


final_text = '\n'.join(cleaned_lines)


final_file_path = 'our_fanfic_cleaned.txt'
with open(final_file_path, 'w') as file:
    file.write(final_text)

final_file_path

'our_fanfic_cleaned.txt'

In [48]:
# chapters_list = clean_text(chapters_list)
# chapters = split_text_into_sections(chapters_list)
# chapters = renumber_chapters_simple(chapters)

In [ ]:
# parts = split_into_parts(chapters_list)
# parts

In [22]:
model.eval()

#final_text = ''
def generate_fanfiction_by_chapters(part):
    #final_text = ''
    #divide part into chapters
    #for part_ in list_content:
        chapters_of_this_part = split_text_into_sections(part)
        text = ''
        for chapter in chapters_of_this_part:
          prompt = f'### Instruction:\nImagine you are a teenage girl who adores books about Harry Potter. \
          You want to write a magical and fantastic fanfiction about Harry Potter. !!!It must be a content for kids!!! \
          Here is a storyline, which you must follow: {part}\n\
          But you must write only the CHAPTER provided in the INPUT. ### Input:\n{chapter}\n\n### Fanfiction:'
          final_chapter = generate_text(prompt, model, tokenizer)
          text += final_chapter
          
          #final_text += final_chapter
          #print(final_chapter)
          return text

In [20]:
f = []
for part in parts:
  chapters_of_this_part = split_text_into_sections(part)
  f
chapters_of_this_part

['Chapter 1 The Enchanted Rose Theo and Pansy visit the magical garden to gather ingredients for a potion They discover a rare enchanted rose that only blooms once a year Theo uses his nonverbal spell skills to help the rose bloom Pansy is amazed by Theos abilities and they share a special moment 2 ',
 'Chapter 2 The Mysterious Plant Theo and Pansy come across a strange mysterious plant that seems to have a mind of its own They work together to figure out how to tame the plant and use it in a potion They learn valuable lessons about teamwork and trust 3 ',
 'Chapter 3 The Secret Garden Theo and Pansy stumble upon a hidden secret garden that is rumored to hold powerful magical creatures They explore the garden and encounter various magical creatures including a friendly phoenix They use their knowledge of potions and transfiguration to help the creatures and learn more about their magical abilities 4 ',
 'Chapter 4 The Magical Feast Theo and Pansy prepare a magical feast using the ingre

In [24]:
len(parts)

8

In [23]:
# final_text = ''
# for i in parts:
#     res = generate_fanfiction_by_chapters(i)
#     final_text += res
#     print(res)

final_text = ''
n = 1
for i in parts:
    res = generate_fanfiction_by_chapters(i)
    res = res.replace('Chapter 1', f'Chapter {n}')
    final_text += res
    n+=1
    print(res)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>
Chapter 1: The Excited Letter

Theo sat at the kitchen table, his eyes fixed on the large, owl-delivered envelope in front of him. His heart raced with excitement as he carefully opened the letter, his breath hitching in his throat as he read the words that confirmed his wildest dreams.

"Congratulations! You have been accepted to Hogwarts School of Witchcraft and Wizardry. You are invited to attend the Sorting Ceremony on September 1st."

Theo's eyes sparkled with joy as he reread the letter, his mind already racing with thoughts of magical classes, new friends, and the thrill of living at Hogwarts. He couldn't believe it, he was finally going to be a real witch or wizard!

He spent the next few days in a state of constant excitement, eagerly anticipating the start of his new life. He spent hours poring over Hogwarts textbooks, learning everything he could about the school and its history. He practiced spells and charms in the garden, trying to perfect his wand movements and incan

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>
Title: The Slytherin's Secret

Chapter 2: The Bully and the Beauty

Theo Malfoy and Pansy Parkinson were two students who stood out in their year at Hogwarts. Theo, with his platinum blonde hair and piercing green eyes, and Pansy, with her fiery red hair and emerald green eyes, were both pure-bloods, a fact that made them targets for many students.

As they walked through the halls of Hogwarts, they could hear the whispers and snickers behind their backs. "Look at those two, so proud of their pure-blood status," one student sneered. "They think they're better than everyone else," another added.

Theo and Pansy tried to ignore the taunts, but it was hard when they saw the disdain in the eyes of their classmates. They found solace in each other's company, and as they spent more time together, they began to develop feelings for one another.

But their happiness was short-lived. The bullying only seemed to intensify, and one day, as they were making their way to their common room, they

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

<s>
As the sun began to set, Theo and Pansy found themselves huddled in the dimly lit corners of the Hogwarts library. The weight of their latest assignment pressing heavily upon their shoulders, they had spent hours poring over ancient texts and scrolls, seeking answers to the complex questions posed by their professor.

But as they delved deeper into the dusty tomes, something unexpected caught their eyes. A faint glimmer of light, barely visible, seemed to emanate from the depths of a hidden passageway. Intrigued, the two students exchanged a glance, and without a word, they set aside their books and crept towards the mysterious opening.

As they ventured further into the passage, the air grew thick with the scent of old parchment and the faint hum of ancient magic. The walls were lined with shelves upon shelves of rare and ancient books, their spines worn and cracked with age. And in the center of the room, there lay a collection of magical artifacts, each one more fascinating than

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
Once upon a sunny afternoon, as Theo and Pansy sat in their favorite spot at Hogwarts library, engrossed in their latest Harry Potter books, a sudden gust of wind swept through the room. The windows rattled, and the candles flickered, as if in anticipation of something extraordinary.

Suddenly, a letter fluttered down from the rafters, landing with a soft thud on the table between them. Theo, always the more curious of the two, picked it up and unfolded it, revealing a beautifully handwritten invitation.

"To Theo and Pansy," it read, "You are cordially invited to a secret meeting at the Three Broomsticks Inn. The time is of the essence, so come prepared to discuss matters of great importance. Yours sincerely, A Friend."

Theo and Pansy exchanged puzzled glances. Who could have sent such a letter? And what could be so important that they needed to keep it a secret? They decided to keep the matter to themselves for the time being and made plans to meet at the Three Broomsticks Inn 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>
Once upon a time, in the mystical land of Hogwarts, there lived two enchanting teenagers named Theo and Pansy. They were both known for their exceptional magical abilities and their deep love for the magical world of Harry Potter.

One sunny afternoon, as they strolled through the winding paths of the Hogwarts grounds, they stumbled upon a hidden entrance to the magical garden. With wide eyes and excited hearts, they stepped inside, eager to explore the wonders that lay before them.

As they wandered deeper into the garden, they came upon a breathtaking sight. There, in the heart of the garden, stood a magnificent enchanted rose. Its petals were the most vibrant shades of pink and red, and it seemed to glow with an inner light.

Theo and Pansy were in awe of the rose's beauty. They approached it cautiously, sensing the powerful magic that surrounded it. As they reached out to touch its velvety petals, they heard a soft voice whisper in their ears.

"The enchanted rose can only be w

In [13]:
for part in parts:
        chapters_of_this_part = split_text_into_sections(part)
chapters_of_this_part

['Chapter 1: The Orchid EncounterTheo surprises Pansy with a beautiful bouquet of orchids, sparking a deeper connection between them.',
 'Chapter 2: The Transfiguration LessonPansy struggles with a Transfiguration spell, and Theo offers to help. Their shared interest in the subject leads to a deeper understanding of each other.',
 'Chapter 3: The Flirtatious GameTheo flirts with Pansy, catching her off guard. She is unsure of how to react, but is touched by his kindness and attention.',
 'Chapter 4: The Secret GardenTheo and Pansy find a hidden garden on school grounds and spend an afternoon together, sharing stories and laughter. Their bond deepens as they connect on a personal level.',
 'Chapter 5: The Late-Night ConfessionAs they walk back to the castle after a late-night study session, Theo confesses his growing feelings for Pansy. She is surprised but touched by his honesty and begins to see him in a new light.']

In [20]:
text

'<s>\n---\n**Chapter 3: The Bullying Begins**\n\nTheo Nott and Pansy Parkinson, both eleven years old, walked hand in hand through the bustling halls of Hogwarts Castle. Their golden-blonde hair shone in the dim light, and their emerald green eyes sparkled with excitement. They were both proud Slytherins, and they wore their house colors with pride.\n\nBut not everyone shared their enthusiasm. As they passed by a group of Gryffindor students, they heard snickers and whispers.\n\n"Look at those Slytherins, so proud of their pure-blood status," one student sneered.\n\n"They think they\'re better than us," another added.\n\nTheo and Pansy ignored the comments and continued walking. But as they entered the Transfiguration classroom, they were met with even more hostility.\n\nMadame Pomfrey, the potions professor, looked up from her desk as they entered. Her expression turned cold as she took in their appearance.\n\n"Slytherins, sit at the back," she ordered, her voice dripping with disdain